<a href="https://colab.research.google.com/github/nakib103/pancancer_classification/blob/master/Residual_LSTM_usingLSTMCell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install 'tensorflow == 2.3.0'

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [ ]:
# import packages

import numpy as np
import pandas as pd
import pickle
import math
import gc

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Bidirectional, Concatenate, Permute, Dot, Input, LSTM, GRU, Multiply
from tensorflow.keras.layers import RepeatVector, Dense, Activation, Lambda, Add
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import LambdaCallback

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
units=32
cell = tf.keras.layers.LSTMCell(units)
input = np.array([[0.2, 0.5], [0.3, 0.7]])
mem = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))
carry = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))
cell(inputs=input, states=[mem, carry])


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



(<tf.Tensor: shape=(2, 32), dtype=float32, numpy=
 array([[-0.01080109, -0.01969664,  0.00749568, -0.00894347, -0.00388242,
         -0.02629492,  0.01472679,  0.01501883, -0.01941722, -0.004275  ,
         -0.0162739 , -0.02795709,  0.00775131,  0.01568452,  0.01625854,
         -0.01623782, -0.00532019, -0.01700117, -0.01727366, -0.03176708,
         -0.02135437, -0.01675564, -0.01587299,  0.00300255, -0.02705372,
         -0.02046608,  0.03093919,  0.01845305, -0.00735357, -0.00407508,
         -0.02986706, -0.0050304 ],
        [-0.01594485, -0.02777299,  0.0094791 , -0.01402404, -0.00457074,
         -0.03634268,  0.021322  ,  0.02014242, -0.0287704 , -0.00649746,
         -0.02201654, -0.03947828,  0.01067325,  0.02266347,  0.02369794,
         -0.02222504, -0.00821367, -0.02402185, -0.02384901, -0.04474057,
         -0.0300826 , -0.02365895, -0.02253139,  0.00374976, -0.03881164,
         -0.03000979,  0.04412488,  0.0254804 , -0.01009233, -0.0068405 ,
         -0.041995  , -0.0

In [ ]:
units=32
mem_init = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))
carry_init = tf.convert_to_tensor(np.zeros((2, units), dtype=np.float32))

input1 = np.array([[0.2, 0.5], [0.3, 0.7]], dtype=np.float32)
mem, carry = tf.keras.layers.LSTMCell(units)(input1, [mem_init, carry_init])


input2 = np.array([[0.2, 0.5], [0.3, 0.7]], dtype=np.float32)
cell2 = tf.keras.layers.LSTMCell(units)(input2, [mem, carry])

In [ ]:
units=32
time_stamp=1
feature=1
mem = tf.keras.layers.Input((1, units), dtype = np.float32)
carry = tf.keras.layers.Input((1, units), dtype = np.float32)

inputs = tf.keras.layers.Input((time_stamp, feature), dtype = np.float32)
outputs = []
for t in range(time_stamp):
  mem0 = mem
  carry0 = carry

  mem, carry = tf.keras.layers.LSTMCell(units)(input1[t:t+1, :], [mem, carry])
  mem = carry[0:]
  carry = carry[1:]

  outputs.append(mem)
  mem = tf.keras.layers.Add()([mem, mem0])

  
model = tf.keras.models.Model(inputs, outputs)
model.summary()

ValueError: ignored